In [10]:
!pip3 install --upgrade kfp

  Using cached https://files.pythonhosted.org/packages/06/cf/a3fa07ee2ef82074e33798b1dff2355ce7d9e477881609687a91da421a7a/kfp-0.1.11-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/c7/99b33c53cf3f20a97a4c4bfd3ab66dcc93d99da0a97cc9597aa36ae6bb62/cryptography-2.4.2-cp34-abi3-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/97/8a/d710f792d6f6ecc089c5e55b66e66c3f2f35516a1ede5a8f54c13350ffb0/requests_toolbelt-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/44/f8286fb7a25a4ff29a4dec1b5baa49571eedc2b2edf6ec4b51e4b511ac0f/kubernetes-8.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/d1/3378cc8184a6524dc92993090ee8b4c03847c567e298305d6cf86987e005/PyJWT-1.6.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/72/53dce9e4f5d6c1aa57b8d408cb34dff1969ecbf10ab7e678f32c5e0e2397/adal-1.2.1-py2.py3-none-any.whl
fairing 0.0.3 has requirement kubernetes=

In [1]:
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp

## Flip Coin Pipeline 

In [12]:
import kfp.dsl as dsl


class RandomNumOp(dsl.ContainerOp):
  """Generate a random number between low and high."""

  def __init__(self, low, high):
    super(RandomNumOp, self).__init__(
      name='Random number',
      image='python:alpine3.6',
      command=['sh', '-c'],
      arguments=['python -c "import random; print(random.randint(%s,%s))" | tee /tmp/output' % (low, high)],
      file_outputs={'output': '/tmp/output'})


class FlipCoinOp(dsl.ContainerOp):
  """Flip a coin and output heads or tails randomly."""

  def __init__(self):
    super(FlipCoinOp, self).__init__(
      name='Flip',
      image='python:alpine3.6',
      command=['sh', '-c'],
      arguments=['python -c "import random; result = \'heads\' if random.randint(0,1) == 0 '
                 'else \'tails\'; print(result)" | tee /tmp/output'],
      file_outputs={'output': '/tmp/output'})


class PrintOp(dsl.ContainerOp):
  """Print a message."""

  def __init__(self, msg):
    super(PrintOp, self).__init__(
      name='Print',
      image='alpine:3.6',
      command=['echo', msg],
  )
    

@dsl.pipeline(
  name='pipeline flip coin',
  description='shows how to use dsl.Condition.'
)
def flipcoin():
  flip = FlipCoinOp()
  with dsl.Condition(flip.output == 'heads'):
    random_num_head = RandomNumOp(0, 9)
    with dsl.Condition(random_num_head.output > 5):
      PrintOp('heads and %s > 5!' % random_num_head.output)
    with dsl.Condition(random_num_head.output <= 5):
      PrintOp('heads and %s <= 5!' % random_num_head.output)

  with dsl.Condition(flip.output == 'tails'):
    random_num_tail = RandomNumOp(10, 19)
    with dsl.Condition(random_num_tail.output > 15):
      PrintOp('tails and %s > 15!' % random_num_tail.output)
    with dsl.Condition(random_num_tail.output <= 15):
      PrintOp('tails and %s <= 15!' % random_num_tail.output)


if __name__ == '__main__':
  import kfp.compiler as compiler
  compiler.Compiler().compile(flipcoin, 'flip_coin4' + '.tar.gz')

In [13]:
client = kfp.Client()

In [14]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2019, 3, 14, 12, 56, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'ca327eef-3dea-45f1-8b17-3617ce175b4d',
                  'name': 'dorons'},
                 {'created_at': datetime.datetime(2019, 3, 23, 12, 39, 16, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'ca903941-7b9a-43c7-8fdb-0f68dce80fa5',
                  'name': 'Run Flip Coins'}],
 'next_page_token': None,
 'total_size': None}

In [16]:
for i in range(3):
    client.run_pipeline(experiment_id='ca327eef-3dea-45f1-8b17-3617ce175b4d',
                    job_name='run_B_{}'.format(i),
                    pipeline_package_path='flip_coin4.tar.gz')

In [17]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2019, 3, 14, 12, 56, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'ca327eef-3dea-45f1-8b17-3617ce175b4d',
                  'name': 'dorons'},
                 {'created_at': datetime.datetime(2019, 3, 23, 12, 39, 16, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'ca903941-7b9a-43c7-8fdb-0f68dce80fa5',
                  'name': 'Run Flip Coins'}],
 'next_page_token': None,
 'total_size': None}